In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np
%matplotlib inline

In [ ]:
# load the raw data
path = '/kaggle/input/mlho-2021/'
X_train = np.load(path + 'X_train.npy')
y_train = np.load(path + 'y_train.npy')
print('shape of the training data:', X_train.shape)
print('shape of the target data:', y_train.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state = 42, train_size = 2520)

In [ ]:
X_test.shape

In [ ]:
# preview some of the training images
plt.figure(figsize=(12,10))
x, y =3, 3
for i in range(9):  
    plt.subplot(y, x, i+1)
    plt.imshow(X_train[i,:,:],cmap='gray')
    plt.title('label: {}'.format(y_train[i]))
    plt.axis('off')
    plt.colorbar()
plt.show()

In [ ]:
import keras

In [ ]:
# analyze data
X_train = X_train.reshape(X_train.shape[0], 150, 150, 1)
y_train = keras.utils.to_categorical(y_train, 4)

In [ ]:
#import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D   # required for CNN
from keras import backend as K

In [ ]:
# train you model
# built a Convolutional Neural Network
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(150,150,1)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))

model.summary()

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 150, 150, 1)
y_train = keras.utils.to_categorical(y_train, 4)

X_test = X_test.reshape(X_test.shape[0], 150, 150, 1)
y_test = keras.utils.to_categorical(y_test, 4)

In [ ]:
X_test.shape

In [ ]:
model_cam = Sequential()
model_cam.add(Conv2D(16,input_shape=(150,150,1),kernel_size=(3,3),activation='relu',padding='same'))
model_cam.add(Conv2D(32,kernel_size=(3,3),activation='relu',padding='same'))
model_cam.add(Conv2D(64,kernel_size=(3,3),activation='relu',padding='same'))
model_cam.add(Conv2D(128,kernel_size=(3,3),activation='relu',padding='same'))
model_cam.add(GlobalAveragePooling2D())
model_cam.add(Dense(4,activation='softmax'))

model_cam.summary()

model_cam.compile(loss=keras.losses.categorical_crossentropy, # no need to use a one hot vector
              metrics=['accuracy'],
              optimizer='adam')

In [ ]:
batch_size = 32
epochs = 20

history = model_cam.fit(X_train, y_train,
           batch_size=batch_size,
           epochs=epochs,
           verbose=1,
           validation_data=(X_test, y_test))

In [ ]:
model_cam.save_weights('CNNCAM_20_epochs_32_batch_Adam.h5')

In [ ]:
batch_size = 32
epochs = 20

history = model_cam.fit(X_train, y_train,
           batch_size=batch_size,
           epochs=epochs,
           verbose=1,
           validation_data=(X_test, y_test))

In [ ]:
# and/or train the model
batch_size = 128
epochs = 40

history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)

In [ ]:
model.save_weights('CNN_40_epochs_128_batch_Adadelta.h5')

In [ ]:
model.load_weights('CNN_40_epochs_128_batch_Adadelta.h5')

In [ ]:
batch_size = 128
epochs = 40

history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)

In [ ]:
model.save_weights('CNN_80_epochs_128_batch_Adadelta.h5')

In [ ]:
batch_size = 128
epochs = 80

history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1)

In [ ]:
# load the test data
X_test = np.load(path + 'X_test.npy')
print('shape of the test data:', X_test.shape)
X_test = X_test.reshape(X_test.shape[0], 150, 150, 1)

In [ ]:
model.save_weights('CNN_160_epochs_128_batch_Adadelta.h5')

In [ ]:
# # confusion matrix
# import seaborn as sns
# from sklearn import metrics

# y_pred = model.predict(X_test)
# # convert one hot to vector
# Y_test = np.argmax(y_test, axis = 1)
# Y_pred = np.argmax(y_pred, axis = 1)

# cm1 = metrics.confusion_matrix(Y_test, Y_pred)

# plt.figure(figsize=(9,9))
# sns.heatmap(cm1, annot=True, fmt=".0f", linewidths=.5, square=True, cmap='Blues_r')
# plt.ylabel('true number', size=17)
# plt.xlabel('predicted number', size=17)
# plt.show()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred.shape

In [ ]:
y_pred[:4]

In [ ]:
Y_pred = np.argmax(y_pred, axis = 1)

In [ ]:
Y_pred

In [ ]:
import pandas as pd

In [ ]:
# make prediction for the test data
df_sample = pd.read_csv(path + 'sample_submission.csv')
df_sample.head()

In [ ]:
df_sub = pd.DataFrame({'Id': range(830), 'Category': Y_pred})

In [ ]:
df_sub.head()

In [ ]:
df_sub.to_csv("submission_one.csv", index=False )

In [ ]:
df_test  = pd.read_csv("submission_one.csv")

In [ ]:
df_test.head()